# Koiter shell (linear) with HHJ

Reference values: $t=0.1$: $0.1856305$, $t=0.01$: $0.1502913$, $t=0.001$: $0.1498749$

In [ ]:
from ngsolve import *
from netgen.csg import *
from netgen.meshing import MeshingStep
from ngsolve.webgui import Draw
import netgen.meshing as meshing

thickness = 0.1
radius    = 1
order = 3

E   = 2.85e4
nu  = 0.3
mu  = E/(2*(1+nu))
lam = E*nu/(1-nu**2)
k   = 5/6


free = "right"
symmetry ="left|top|bottom"

zeta = IfPos(y, atan(z/y), pi/2)
A = sqrt(1+y**2)
B = sqrt(1+2*y**2)
e_n = -specialcf.normal(3)
force = 1e4*thickness*cos(2*zeta)*e_n*thickness**2

mapping = lambda x,y,z : (x,sqrt(1+x**2)*cos(pi/2*y),sqrt(1+x**2)*sin(pi/2*y))
geom = meshing.SurfaceGeometry(mapping)
mesh = Mesh(geom.GenerateMesh(quads=False, nx=5, ny=5)).Curve(order)
Draw(mesh)

poi = { "point" : (0,0,radius, BND), "component" : 2 }

In [ ]:
interpolateMembrane=True
def MaterialStress(mat):
    return E/(1-nu**2)*((1-nu)*mat+nu*Trace(mat)*Id(3))

def MaterialStressInv(mat):
    return (1+nu)/E*(mat-nu/(nu+1)*Trace(mat)*Id(3))

G = E/(2*(1+nu))
t = thickness
fesU = VectorH1(mesh, order=order, dirichletx_bbnd="left", dirichlety_bbnd="top", dirichletz_bbnd="bottom")
fesS = HDivDivSurface(mesh, order=order-1, discontinuous=True)
fesH = NormalFacetSurface(mesh, order=order-1, dirichlet_bbnd="left|top|bottom")
        
fes = fesU*fesS*fesH
(u,sigma,uh), (du,dsigma,duh) = fes.TnT()
sigma,dsigma,uh,duh = sigma.Trace(),dsigma.Trace(),uh.Trace(),duh.Trace()

fesRR = HCurlCurl(mesh, order=order-1)

n       = specialcf.normal(3)
tangent = specialcf.tangential(3)
nel     = Cross(n,tangent)

Ptau = Id(3) - OuterProduct(n,n)
gradu = Grad(u).Trace()
graddu = Grad(du).Trace()

solution = GridFunction(fes)

Hn = CF( (u.Operator("hesseboundary").trans*n), dims=(3,3) )
dHn = CF( (du.Operator("hesseboundary").trans*n), dims=(3,3) )

a = BilinearForm(fes, symmetric=True, condense=True)
# membrane part
if interpolateMembrane:
    a += (t*InnerProduct(MaterialStress(Interpolate(Sym(Ptau*gradu),fesRR)), Interpolate(Sym(Ptau*graddu),fesRR)))*ds
else:
    a += (t*InnerProduct(MaterialStress(Sym(Ptau*gradu)), Sym(Ptau*graddu)))*ds
#bending part
a += (-12/t**3*InnerProduct(MaterialStressInv(sigma),dsigma))*ds
a += (InnerProduct(dsigma,Hn) + InnerProduct(sigma,dHn))*ds
a += ( -(sigma*nel*nel)*(graddu.trans*n*nel) - (dsigma*nel*nel)*(gradu.trans*n*nel) + (uh*nel)*(dsigma*nel)*nel + (duh*nel)*(sigma*nel)*nel )*ds(element_boundary=True)


f = LinearForm(fes)
f += force*du*ds

In [ ]:
with TaskManager():
    a.Assemble()
    f.Assemble()
    
    r = f.vec.CreateVector()
    if a.condense:     
        r.data = f.vec
        r.data += a.harmonic_extension_trans * r
        inv = a.mat.Inverse(fes.FreeDofs(True), inverse="sparsecholesky")
        solution.vec.data = inv*r
        solution.vec.data += a.harmonic_extension * solution.vec
        solution.vec.data += a.inner_solve * r
    else:
        inv = a.mat.Inverse(self.fes.FreeDofs(), inverse="umfpack")
        r.data = f.vec
        solution.vec.data = inv*r

In [ ]:
gfu,gfbeta,gfuh = solution.components
print(gfu(mesh(0,0,1,BND))[2])
Draw(gfu,mesh)